In [1]:
%matplotlib inline
if __package__ is None:
    import sys, os
    sys.path.append(os.path.realpath("/data/shared/Software/"))
from CMS_Deep_Learning.storage.archiving import *
from CMS_Deep_Learning.layers.slice import Slice
from CMS_Deep_Learning.layers.lorentz import Lorentz
from keras.utils.visualize_util import plot
from IPython.display import Image, display
from CMS_Deep_Learning.postprocessing.colors import colors_contrasting
from CMS_Deep_Learning.postprocessing.analysistools import *
from CMS_Deep_Learning.postprocessing.plot import *
from CMS_Deep_Learning.postprocessing.metrics import *
import numpy as np
archive_dir = "/data/shared/Delphes/keras_archive/"

#def sortTrialsOn(lst):
    

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
trials = get_trials_by_name("LSTM", archive_dir)
#trials = findWithMetrics(trials, {"useObjTypeColumn" : True})
trials = [t for t in trials if t.is_complete()]
for trial in trials:
    trial.summary(showTraining=False,showValidation=False, showFit=False, showCompilation=False)
print("TotalNumber of Trials:", len(trials)) 

--------------------------------------------------
TRIAL SUMMARY (1d645dbd33ca1a4ec695e8cc49d4a4b6f848efda)
    Record_Info:
        name = [u'LSTM'], elapse_time = 4:14:32
        val_acc = 0.7873
        num_train = 150000
        activation = tanh, depth = 1, dropout = 0.0, labels = [u'qcd', u'ttbar'], lstm_dropout = 0.0, num_val = 150000, output_activation = softmax, patience = 8, query = None, sort_on = Eta, useObjTypeColumn = True
--------------------------------------------------
--------------------------------------------------
TRIAL SUMMARY (0131079e4a5cbf5fba3ddcfbf3182f8e10c4fb17)
    Record_Info:
        name = [u'LSTM'], elapse_time = 6:41:01
        val_acc = 0.8388
        num_train = 225000
        activation = tanh, depth = 1, dropout = 0.0, labels = [u'ttbar', u'wjet', u'qcd'], lstm_dropout = 0.0, num_val = 225000, output_activation = softmax, patience = 8, query = None, sort_on = Eta, useObjTypeColumn = True
--------------------------------------------------
-------

In [5]:
import sys
import numpy as np
from CMS_Deep_Learning.storage.archiving import DataProcedure
if(sys.version_info[0] > 2):
    from inspect import signature
    getNumParams = lambda f: len(signature(f).parameters)
else:
    from inspect import getargspec
    getNumParams = lambda f: len(getargspec(f)[0])


class DataIterator:
    def __init__(self, proc, num_samples=None, return_X=False, return_Y=True, accumilate=None, prediction_model=None):
        if (isinstance(proc, list)):
            first_data = proc[0].getData()
        else:
            first_data = proc.getData()
        if (isinstance(first_data, types.GeneratorType)):
            proc = first_data

        self.proc = proc
        self.num_samples = num_samples
        self.accumilate = accumilate
        self.prediction_model = prediction_model
        self.return_X = return_X
        self.return_Y = return_Y
        if (isinstance(proc, list)):
            if (False in [isinstance(p, DataProcedure) for p in proc]):
                raise ValueError("procedure list must contain only DataProcedures")
            # self.next = _listNext
            self.proc_itr = iter(self.proc)
            self.mode = "list"
        elif (isinstance(proc, types.GeneratorType)):
            if (num_samples == None):
                raise ValueError("num_samples must be passed along with procedure generator.")
            # self.next = _genNext
            self.mode = "generator"
        else:
            raise ValueError("Bad input.")
            # initialize(proc, num_samples=num_samples, accumilate=accumilate, prediction_model=prediction_model)

    def getLength(self):
        if (self.num_samples == None):
            num_samples = 0
            for p in self.proc:
                if (isinstance(p, DataProcedure)):
                    X, Y = p.getData()
                else:
                    X, Y = p
                if (not isinstance(Y, list)): Y = [Y]
                num_samples += Y[0].shape[0]
            self.num_samples = num_samples
        return self.num_samples

    def asList(self):
        if (self.accumilate != None): num_params  =getNumParams(self.accumilate)
        X_out = None
        Y_out = None
        pred_out = None
        acc_out = None
        pos = 0
        for p in self.proc:
            X, Y = p.getData()

            if (not isinstance(Y, list)): Y = [Y]
            L = Y[0].shape[0]

            if (not isinstance(X, list)): X = [X]
            if (self.return_X):
                if (X_out == None): X_out = [[None] * self.getLength() for i in range(len(X))]
                # print([len(x) for x in X_out])
                # print([len(x) for x in X])
                for i, x in enumerate(X):
                    Xi_out = X_out[i]
                    # print(len(xi))
                    for j in range(L):
                        Xi_out[pos + j] = x[j]

            if (self.return_Y):
                if (Y_out == None): Y_out = [[None] * self.getLength() for i in range(len(Y))]
                for i, y in enumerate(Y):
                    Yi_out = Y_out[i]
                    for j in range(L):
                        Yi_out[pos + j] = y[j]

            if (self.prediction_model != None):
                if (pred_out == None): pred_out = [None] * self.getLength()
                pred = self.prediction_model.predict_on_batch(X)
                for j in range(L):
                    pred_out[pos + j] = pred[j]

            if (self.accumilate != None):
                if (acc_out == None): acc_out = [None] * self.getLength()
                if(num_params == 1):
                    acc = self.accumilate(X)
                else:
                    acc = self.accumilate(X,Y)
                for j in range(L):
                    acc_out[pos + j] = acc[j]
                    print(len(acc_out),pos + j)

            pos += L
            print(pos, self.accumilate)  # ,acc_out))
        out = []
        if (X_out != None):
            for i, xo in enumerate(X_out):
                X_out[i] = np.array(xo)
            out.append(X_out)
        if (Y_out != None):
            for i, yo in enumerate(Y_out):
                Y_out[i] = np.array(yo)
            out.append(Y_out)
        if (pred_out != None):
            out.append(np.array(pred_out))
        if (acc_out != None):
            out.append(np.array(acc_out))
        return out

    # def _genNext():
    #    #N = num_samples/ba
    #    data = data.getData()
    #    for i in range(self.num_samples)
    #        yield next(data)
    #    return StopIteration()
    '''
    def _listNext():
        for p in self.proc:
            X,Y = p.getData()
            pred = self.prediction_model.predict_on_batch(X) if self.prediction_model != None else None
            acc = self.accumilate(X) if self.accumilate != None else None
            for  in
                yield next(self.proc)
        return StopIteration()
    '''

    def __iter__(self):
        return self

        # def next(self):
        #    self.count += 1
        #    if(self.mode == 0):


class TrialIterator(DataIterator):
    def __init__(self, trial, data_type="val", return_X=False, return_Y=True, accumilate=None, return_prediction=False,
                 custom_objects={}):
        if (data_type == "val"):
            proc = [DataProcedure.from_json(trial.archive_dir, t) for t in trial.val_procedure]
            num_samples = trial.nb_val_samples
        elif (data_type == "train"):
            proc = [DataProcedure.from_json(trial.archive_dir, t) for t in trial.train_procedure]
            num_samples = trial.samples_per_epoch
        else:
            raise ValueError("data_type must be either val or train but got %r" % data_type)
        model = None
        if (return_prediction):
            model = trial.compile(loadweights=True, custom_objects=custom_objects)
        DataIterator.__init__(self, proc, num_samples=num_samples, return_X=return_X, return_Y=return_Y,
                              accumilate=accumilate, prediction_model=model)
def getError(model, data=None, num_samples=None,custom_objects={}, ignoreAssert=False):
    '''
    Finds the standard error of the mean for the validation accuracy of a model on a dataset or a trial.
    #Arguements:
            model -- The model being evaluated, or a KerasTrial containing a valid model.
            data  -- A generator, or DataProcedure containing the data to be run through the model. If a generator or DataProcedure
                     containing a generator is given the num_samples must be set. If model is a KerasTrial this can be set to None, and the validation
                     set will be found from the archive (or computed) and used in place of data.
            num_samples -- The number of samples to evaluate the error on.
            custom_objects -- A dictionary keyed by names containing the classes of any model components not used in the standard Keras library.
            ignoreAssert -- If True ignore assertion errors. This code tests to see that the validation accuracy it computes is similar to the one computed by keras.
                            If this is not the case then an error will be raised.
    #Returns:
        The standard error of the validation accuracy
    '''
    isTrial = False
    if(isinstance(model, KerasTrial)):
        trial = model
        model = trial.compile(loadweights=True,custom_objects=custom_objects)
        isTrial = True

    def accum(X,Y):
        return model.test_on_batch(X,Y)

    if (isTrial):
        dItr = TrialIterator(trial, return_X=False, return_Y=False, return_prediction=False, accumilate=accum)
    else:
        dItr = DataIterator(data, return_X=False, return_Y=False,
                            num_samples=num_samples, accumilate=accum)

    batch_metrics = dItr.asList()

    batch_metrics = np.array(batch_metrics)
    avg = np.mean(batch_metrics, axis=0, dtype='float64')
    sem = np.std(batch_metrics, axis=0, dtype='float64')/np.sqrt(i)
    if(not ignoreAssert and trial.get_from_record("val_acc") != None):
        np.testing.assert_almost_equal(trial.get_from_record("val_acc"), avg[1], decimal=3)
    else:
        trial.to_record({"val_acc_" : avg[1]})
    trial.to_record({"val_acc_error" : sem[1]})
    return trial.get_from_record("val_acc_error")

trial = trials[0]
print(getError(trial,custom_objects={"Slice": Slice, "Lorentz": Lorentz}))

DataProcedure results '3d5a9733451c92cb0218483c196cbfdddbfef850' read from archive
DataProcedure results '3d5a9733451c92cb0218483c196cbfdddbfef850' read from archive
(40000, 0)
(40000, 1)


IndexError: list index out of range

In [ ]:
#import sklearn

trial = trials[0]

observable_ordering=['E/c', 'Px', 'Py', 'Pz', 'PT_ET','Eta', 'Phi', 'Charge', 'X', 'Y', 'Z', 
                     'Dxy', 'Ehad', 'Eem', 'MuIso', 'EleIso', 'ChHadIso','NeuHadIso','GammaIso']
object_ordering=["Electron", "MuonTight", "MissingET","EFlowPhoton", "EFlowNeutralHadron", "EFlowTrack"]

#=d = accVsEventChar(trial, None, np.sum, "PT_ET", objects=["EFlowPhoton","EFlowNeutralHadron","EFlowTrack"], 
#                       observable_ordering=observable_ordering,object_ordering=object_ordering,
#                       bins=20,custom_objects={"Slice": Slice, "Lorentz": Lorentz},equalBins=False)
d = accVsEventChar(trial, None, np.sum, "PT_ET", objects=["EFlowPhoton","EFlowNeutralHadron","EFlowTrack"],
                       observable_ordering=observable_ordering,object_ordering=object_ordering,
                       bins=20,custom_objects={"Slice": Slice, "Lorentz": Lorentz},equalBins=False, plot=True)
#assert set([t for t in d.items()]) == set([t for t in d2.items()])
#def plot_ROC(trial):
#    X,Y = 
#    sklearn.metrics.roc_curve(y_true, y_score, pos_label=None, sample_weight=None, drop_intermediate=True)

In [ ]:
trials = findWithMetrics(trials, {"useObjTypeColumn" : True})
print_by_labels(trials, 4)

#plot_history([("Fully connected:", trials[0].get_history())], plotLoss=False)


In [ ]:
t = findWithMetrics(trials, {"labels" : [u'ttbar', u'wjet', u'qcd'], 'sort_on' : "Phi"})
t[0].summary(showCompilation=True, showFit=True)

In [ ]:
plotEverything(trials)
    
    


In [ ]:
phi = findWithMetrics(trials,{"sort_on" : "Phi"})
eta = findWithMetrics(trials,{"sort_on" : "Eta"})
PT = findWithMetrics(trials,{"sort_on" : "PT_ET"})
sortOnMetric(phi, "labels")
sortOnMetric(eta, "labels")
sortOnMetric(PT, "labels")
labelGroups = zip(phi,eta,PT)

In [ ]:
colors = [(0,0,1.0),(.25,.75,.25), (1,.65,0)]
names = ["Phi", "Eta","PT"]
lims = [[.7,.91], [.8,1.0],  [.8,1.0],  [.4,.82]]
for j , tup in enumerate(labelGroups):
    plots = []
    for i, b in enumerate(tup):
        labels = b.get_from_record("labels")
        if(labels == None): labels = b.get_from_record("lables")
        title = str(tuple([str(x) for x in labels])) if(labels != None) else "Cannot Find Labels"
        title = title + " Accuracy vs Epoch"
        name = names[i]
        model = b.get_model(custom_objects={"Slice":Slice, "Lorentz" : Lorentz})
        history = b.get_history()
        color = colors[i]
        plots.append((name, history, color))
    plot_history(plots, plotLoss=False, title=title, acclims=lims[j])
    
    

In [ ]:
data = np.zeros(( len(labelGroups[0]),len(labelGroups) ) ).tolist()
print(data)
columns = [None] * len(labelGroups)
rows = [" "+ n + " " for n in names]

for j , tup in enumerate(labelGroups):
    for i, b in enumerate(tup):
        labels = b.get_from_record("labels")
        columns[j] = str(tuple([str(x) for x in labels]))
        error = getError(b,custom_objects={"Slice":Slice, "Lorentz" : Lorentz}, ignoreAssert=True)
        d = "%.5f %s %.5f" % (b.get_from_record("val_acc"),unichr(177), error)
        data[i][j] = d
        
        
#print(data)
        

In [ ]:
plotTable(rows, columns, data,scale=2, title="Validation Accuracy for LSTM Trials by Classification and Sorting")

In [ ]:
def getTrialBins(trial, bins=20):
    trial.summary()
    d = accVsEventChar(trial, None, np.sum, "PT_ET", ["EFlowPhoton","EFlowNeutralHadron","EFlowTrack"],
                       bins=bins,custom_objects={"Slice": Slice, "Lorentz": Lorentz},equalBins=False)
    #plotBins(d,title='Accuracy vs Sum of PF Candidate PT', xlabel="PT GeV", ylabel='Accuracy', color=(0.553,0.188,0.38))
    return d
eta_bins = [getTrialBins(t,40) for t in eta]
phi_bins = [getTrialBins(t,40) for t in phi]

In [ ]:
colors = [(0,0,1.0),(.25,.75,.25), (1,0,0), (1,.65,0)]
lables = [trial.get_from_record("labels") for trial in eta]
plotBins(eta_bins,mode="scatter",title='Eta-LSTM: Accuracy vs Sum of PF Candidate PT',binLabels=lables, xlabel="PT GeV", ylabel='Accuracy',
         legendTitle="Classification",colors=colors, alpha=.2, ylim=(0.75, 1.025), xlim=(0,3000))

lables = [trial.get_from_record("labels") for trial in phi]
plotBins(phi_bins,mode="scatter",title='Phi-LSTM: Accuracy vs Sum of PF Candidate PT',binLabels=lables, xlabel="PT GeV", ylabel='Accuracy',
         legendTitle="Classification",legendBelow=True,colors=colors, alpha=.4, ylim=(0.75, 1.025), xlim=(0,3000))